## Inflation data

Get the month on month inflation data from stat.gov.pl




In [5]:
from requests import get
from csv import reader
from datetime import date
import pandas as pd


In [6]:
data, final_data = {}, {}
url = 'https://stat.gov.pl/download/gfx/portalinformacyjny/pl/defaultstronaopisowa/4741/1/1/miesieczne_wskazniki_cen_towarow_i_uslug_konsumpcyjnych_od_1982_roku.csv'
content = get(url).content.decode('windows-1250').replace(',', '.').replace(';', ',')
lines = list(reader(content.split('\n')))[1:-1]
for record in lines:
    _, _, method, year, month, value, _, _, _ = record
    if method != 'Poprzedni miesiąc = 100': continue
    record_date = date(int(year), int(month), 1)
    if value == '': continue
    data[record_date] = float(value)

data
df = pd.DataFrame(data.items(), columns=['Date', 'Value'])

# remove rows with missing values 
df.dropna(inplace=True)
# sort by date descending
df.sort_values(by='Date', ascending=True, inplace=True)
df.set_index('Date', inplace=True)

df['inter'] = (df['Value']/100)-1

df['last_12m'] = (df['inter']).rolling(window=12).sum() * 100 + 100
df.sort_values(by='Date', ascending=False, inplace=True)
df.head(40)


,Value,inter,last_12m
Date,,,
2023-09-01,99.6,-0.004,107.5
2023-08-01,100.0,0.000,109.5
2023-07-01,99.8,-0.002,110.3
2023-06-01,100.0,0.000,111.0
2023-05-01,100.0,0.000,112.5
2023-04-01,100.7,0.007,114.2
2023-03-01,101.1,0.011,115.5
2023-02-01,101.2,0.012,117.7
2023-01-01,102.5,0.025,116.2


## Inflation data

Get the month on month inflation data from stat.gov.pl




In [7]:
# scatter plot with index date and value

import plotly.express as ex

fig = ex.scatter(df, x=df.index, y='Value', title='CPI', labels={'x':'Date', 'y':'Value'})
# fig.add_scatter(df, x=df.index, y=df['last_12m'], mode='lines', name='Last 12 months')
fig.add_scatter(x=df.index, y=df['last_12m'], mode='lines', name='Last 12 months')
fig.show()


